# 0. Introduction

**Part 1:** 

* [🐝 Weights & Biases Tutorial (beginner)](https://www.kaggle.com/code/samuelcortinhas/weights-biases-tutorial-beginner)

<br>

<center>
<img src='https://i.postimg.cc/P5gnLhGY/tuning-methods.png' width=650>
</center>

<br>

This notebook is a tutorial on how to carry out **hyper-parameter tuning** via **sweeps** in **Weights and Biases**. This is a great technique to learn as it will automatically find the right combination of hyper-parameters that leads to the model with the **best score**. We will cover:

* *Random search*
* *Grid search*
* *Bayesian optimisation*

all while being **tracked** to the WandB dashboard.

# 1. Libraries

In [1]:
# Core
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.4)
import matplotlib.pyplot as plt
%matplotlib inline

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

# Tensorflow
import tensorflow as tf

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# This turns off wandb logs
import os
#os.environ['WANDB_SILENT'] = 'true'

# 2. WandB

The first step is to **log-in** to your wandb account. Find a **step-by-step guide** here: [🐝 Weights & Biases Tutorial (beginner)](https://www.kaggle.com/code/samuelcortinhas/weights-biases-tutorial-beginner)

In [2]:
import wandb

In [3]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb_api_key")

wandb.login(key=my_secret)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# 3. GPU

Connect PyTorch to GPU to accelerate training.

In [4]:
# Config device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# 4. Data

Download **MNIST digit classification** dataset from tensorflow.

In [5]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalise and reshape arrays
X_train, X_test = X_train.reshape((-1, 784))/255.0, X_test.reshape((-1, 784))/255.0

# Print shapes
X_train.shape, X_test.shape

11501568/11490434 [==============================] - 1s 0us/step


((60000, 784), (10000, 784))

# 5. PyTorch dataset

Construct a PyTorch dataset that returns **training/testing samples**. If you are new to PyTorch, check out my other tutorials: [PT3 - Neural Networks with PyTorch](https://www.kaggle.com/code/samuelcortinhas/pt3-neural-networks-with-pytorch)

In [6]:
# Create dataset
class MNIST(Dataset):
    # Initialise
    def __init__(self, subset='train'):
        super().__init__()
        self.subset = subset
        
        # Store data
        if self.subset=='train':
            self.X = torch.from_numpy(X_train.astype(np.float32))
            self.y = torch.from_numpy(y_train)
        elif self.subset=='test':
            self.X = torch.from_numpy(X_test.astype(np.float32))
            self.y = torch.from_numpy(y_test)
        else:
            raise Exception("subset must be train or test")
    
    # Get item in position given by index
    def __getitem__(self, index):
        return self.X[index], self.y[index]
        
    # Length of dataset 
    def __len__(self):
        return self.X.shape[0]

In [7]:
# Datasets
train_dataset = MNIST(subset='train')
test_dataset = MNIST(subset='test')

# 6. Model

Construct model with hyper-parameters as input.

In [8]:
# Feedforward neural network
class NeuralNet(nn.Module):
    def __init__(self, layer1_size, layer2_size, dropout_rate):
        super().__init__()
        
        # Layers
        self.lin1 = nn.Linear(in_features=784, out_features=layer1_size)
        self.lin2 = nn.Linear(in_features=layer1_size, out_features=layer2_size)
        self.lin3 = nn.Linear(in_features=layer2_size, out_features=10)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p=dropout_rate)
        
    def forward(self, x):
        
        # Hidden layer 1
        out = self.lin1(x)
        out = self.relu(out)
        out = self.drop(out)
        
        # Hidden layer 2
        out = self.lin2(out)
        out = self.relu(out)
        out = self.drop(out)
        
        # Output layer (no softmax needed)
        out = self.lin3(out)
        
        return out

# 7. Helper functions

Create functions to do **one epoch of training/validation**.

In [9]:
def train_one_epoch(train_loader, model, loss, optimiser):
    # Track loss
    loss_epoch = 0
    
    # Loop over minibatches
    for imgs, labels in train_loader:
        # Send to device
        imgs = imgs.to(device)
        labels = labels.to(device)

        # Forward pass
        preds = model(imgs)
        L = loss(preds,labels)

        # Backprop
        L.backward()

        # Update parameters
        optimiser.step()

        # Zero gradients
        optimiser.zero_grad()

        # Track loss
        loss_epoch += L.detach().item()
        
    return loss_epoch/len(train_loader)

In [10]:
def evaluate_one_epoch(test_loader, model, loss):
    # Track loss
    val_loss_epoch = 0
    
    # Don't update weights
    with torch.no_grad():
        # Validate
        for val_imgs, val_labels in test_loader:
            # Reshape
            val_imgs = val_imgs.to(device)
            val_labels = val_labels.to(device)

            # Forward pass
            val_preds = model(val_imgs)
            val_L = loss(val_preds,val_labels)

            # Track loss
            val_loss_epoch += val_L.item()
            
    return val_loss_epoch/len(test_loader)

# 8. Training function

To carry out a wandb sweep, we need to provide the agent a **training function**. Inside this function, we have to **initialise a run**, **do the training** and finally **log metrics** to the dashboard. 

Note that the hyper-parameters are defined from `wandb.config` instead of hard-coded values so that the agent can **change these every run**.

In [11]:
def main(verbose=False):
    # Initialise run
    run = wandb.init()
    
    # Dataloaders
    train_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=wandb.config.batch_size, shuffle=False)
    
    # Build model
    model = NeuralNet(wandb.config.layer1_size, wandb.config.layer2_size, wandb.config.dropout_rate).to(device)
    
    # Define loss and optimiser
    loss = nn.CrossEntropyLoss()
    optimiser = optim.Adam(params=model.parameters(), lr=wandb.config.learning_rate)

    # Loop over epochs
    for epoch in range(wandb.config.n_epochs):
        
        # Train
        train_loss = train_one_epoch(train_loader, model, loss, optimiser)
        
        # Evaluate
        val_loss = evaluate_one_epoch(test_loader, model, loss)

        # Log metrics
        wandb.log({
        'epoch': epoch, 
        'train_loss': train_loss,
        'val_loss': val_loss
      })

        # Print loss
        if verbose:
            if (epoch+1)%5==0:
                print(f'Epoch {epoch+1}/{wandb.config.n_epochs}, loss {train_loss:.5f}, val_loss {val_loss:.5f}')

# 9. Sweep config

The config file defines how the sweep is carried out. It has the structure:

* `method` : (required) The search strategy. This can be `random`, `grid` or `bayes`.
* `name` : (optional) Name of the sweep that appears in the dashboard. Different sweeps should have different names.
* `metric` : (required for some search stragegies) Metric to optimise. 
* `parameters` : (required) Hyper-parameter bounds to search.
* `early_terminate` : (optional) Early stopping criteria.

For more details, check the [documentation](https://docs.wandb.ai/guides/sweeps/define-sweep-configuration).

In [12]:
sweep_configuration = {
    'method': 'bayes',  # random, grid or bayes
    'name': 'sweep-bayes',
    'metric': {'goal': 'minimize', 'name': 'val_loss'},
    'parameters': 
    {
        'batch_size': {'values': [64, 128, 256]},
        'n_epochs': {'values': [10, 20, 30]},
        'learning_rate': {'max': 0.1, 'min': 0.0001},
        'layer1_size': {'values': [64, 128, 256]},
        'layer2_size': {'values': [64, 128, 256]},
        'dropout_rate': {'max': 0.5, 'min': 0.1}
     }
}

# 10. Run sweep

We first generate the **sweep id** using the sweep configuration. You can also specify the `entity` (username or team name) and `project` (project name). 

In [13]:
sweep_id = wandb.sweep(sweep=sweep_configuration, entity='scortinhas', project='mnist-sweep')

Create sweep with ID: 8tkdrrub
Sweep URL: https://wandb.ai/scortinhas/mnist-sweep/sweeps/8tkdrrub


Then we use the `wandb.agent` API call to start a Weights & Biases sweep. Specify how the maximum number of runs using `count`.

In [14]:
# Maximum 'count' runs
wandb.agent(sweep_id, function=main, count=30)

wandb: Agent Starting Run: mu5ep2ii with config:
wandb: 	batch_size: 64
wandb: 	dropout_rate: 0.4226852991689247
wandb: 	layer1_size: 256
wandb: 	layer2_size: 64
wandb: 	learning_rate: 0.0588645534728463
wandb: 	n_epochs: 10
wandb: Currently logged in as: scortinhas. Use `wandb login --relogin` to force relogin


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,▁▆▅█▇▇▇▇▇▇
val_loss,▅▁▆▆█▇██▇█
epoch,9
train_loss,2.30662
val_loss,2.30467


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 9mrwenai with config:
wandb: 	batch_size: 64
wandb: 	dropout_rate: 0.12157851413221442
wandb: 	layer1_size: 64
wandb: 	layer2_size: 128
wandb: 	learning_rate: 0.017608239390323722
wandb: 	n_epochs: 20


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,█▅▄▄▃▃▃▂▂▂▂▂▂▁▁▂▁▁▁▁
val_loss,▅▂▁▂▁▂▁▄▂▃▄▂▁▄▁▂▅▄█▅
epoch,19
train_loss,0.25808
val_loss,0.37296


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: a8zeclp9 with config:
wandb: 	batch_size: 256
wandb: 	dropout_rate: 0.23846199258695655
wandb: 	layer1_size: 64
wandb: 	layer2_size: 128
wandb: 	learning_rate: 0.020684658837290627
wandb: 	n_epochs: 20


...

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,█▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▃▃▄▂▂▁▂▁▂▃▂▂▂▂▃▂▃▂
epoch,19
train_loss,0.09779
val_loss,0.15329


wandb: Agent Starting Run: 64hbzhlh with config:
wandb: 	batch_size: 256
wandb: 	dropout_rate: 0.16062445313501775
wandb: 	layer1_size: 64
wandb: 	layer2_size: 128
wandb: 	learning_rate: 0.0003535916242357882
wandb: 	n_epochs: 20


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,█▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▅▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch,19
train_loss,0.09564
val_loss,0.1206


wandb: Agent Starting Run: 4sw6viuz with config:
wandb: 	batch_size: 256
wandb: 	dropout_rate: 0.21036839471140145
wandb: 	layer1_size: 64
wandb: 	layer2_size: 64
wandb: 	learning_rate: 0.0030337343955669223
wandb: 	n_epochs: 30


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_loss,█▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▃▃▃▂▂▂▁▂▁▂▂▂▂▂▂▁▂▁▂▁▂▁▂▂▂▁▁▂
epoch,29
train_loss,0.07601
val_loss,0.13786


# 11. Dashboard


Now we can go to our dashboard and view the results. To view my (public) dashboard for this project go to: [https://wandb.ai/scortinhas/mnist-sweep/sweeps/mt38qsqf?workspace=user-scortinhas](https://wandb.ai/scortinhas/mnist-sweep/sweeps/mt38qsqf?workspace=user-scortinhas).

<br>

<center>
<img src='https://i.postimg.cc/76jr8kxK/sweep-dash.png' width=650>
</center>

<br>

The dashboard stores the history for each individual run on the left panel. In the center, we have **customisable and interactive panels** that summarise the sweep. For example, **loss plots**, **parameter importances** and **parallel plots**. These give great insight into how different parameters are affecting the final score, postively or negatively. 

Finally, you can find the **best run** by navigating to the **sweep table** on the left and order runs by the desired metric. 